In [1]:
import pandas as pd
import pyam
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
results_path = '../../../data/TOD/model_results/to_share/model_results_to_share_Outlooks.xlsx'

In [3]:
df_world = pd.read_excel(results_path)

scenario_rename_dict = {
    "REF": "Current Trends",
    "LIFE-TP": "Behaviour-focused",
    "TECH-TP": "Technology-focused",
    "REF-v2": "Current Trends",
    "LIFE-TP-v2": "Behaviour-focused",
    "TECH-TP-v2": "Technology-focused",
}


df_world = df_world[df_world['region']=='World'].reset_index(drop=True)
df_world = df_world.rename(columns={'topic':'model'})
df_world.scenario = df_world.scenario.replace(scenario_rename_dict)
df_world.model = 'SPM'


df_world = pyam.IamDataFrame(df_world)
df_world.rename(scenario=scenario_rename_dict, inplace=True)
df_world = df_world.filter(year=[2020, 2050])


df_world.subtract('Primary Energy|Biomass', "Primary Energy|Biomass|Traditional", 'Primary Energy|Biomass|Modern', append=True)
df_world.aggregate("Primary Energy|Renewables", ["Primary Energy|Non-Biomass Renewables",  'Primary Energy|Biomass|Modern'], append=True)

df_world.divide("Primary Energy|Renewables",
                "Primary Energy", 
                "Primary Energy|Renewables (Fraction)", 
                ignore_units=True, 
                append=True)
df_world.multiply("Primary Energy|Renewables (Fraction)", 
            100, 
            "Primary Energy|Renewables (Share)", 
            ignore_units='%', 
            append=True)

df_world.divide('Emissions|Kyoto Gases', 
                'Primary Energy', 
                'Decarbonization Rate (Kyoto Gases)', 
                ignore_units=True, 
                append=True)

df_world.divide('Emissions|CO2', 
                'Primary Energy', 
                'Decarbonization Rate (CO2)', 
                ignore_units=True, 
                append=True)

df_world.divide('Final Energy|Electricity', 
                'Final Energy', 
                'Electrification Rate', 
                ignore_units=True, 
                append=True)


df_world.aggregate("Material Extraction", append=True)
df_world.divide('Primary Energy', 'GDP|PPP', 'Energy Intensity', append=True, ignore_units=True)


# df_income = df_income.filter(variable='Food Demand', keep=False)
df_world.filter(variable='Energy Intensity').timeseries()

2020      2050
model scenario           region variable         unit                       
SPM   Behaviour-focused  World  Energy Intensity unknown  0.005667  0.001368
      Current Trends     World  Energy Intensity unknown  0.005676  0.003460
      Technology-focused World  Energy Intensity unknown  0.005657  0.002167

In [4]:
variable_list = [
    'Energy Intensity',
    'Primary Energy|Renewables (Share)', #?
    'Yield|Cereal',
    'Food Demand|Crops (Share)',
    'Land Cover|Forest and Other Natural Land',
    # Aggregate material extraction to one
    'Material Extraction',
    'Decarbonization Rate (Kyoto Gases)',
    'Decarbonization Rate (CO2)',
    'Electrification Rate',
    ]

df_acc = df_world.filter(variable=variable_list)

df_2020 = df_acc.filter(year=2020).timeseries()[2020]
df_2050 = df_acc.filter(year=2050).timeseries()[2050]
df_yearly_result = abs(((df_2050/df_2020)**(1/30)-1)*100)
df_yearly_result = df_yearly_result.reset_index()
df_yearly_result.unit = '%'
df_yearly_result = df_yearly_result.rename(columns={0:'yearly change'})

df_result = ((df_2050-df_2020)/df_2020)*100
df_result = df_result.reset_index()
df_result.unit = '%'
df_result = df_result.rename(columns={0:'change'})
df_result['yearly change'] = df_yearly_result['yearly change']
df_result[2020] = df_2020.reset_index()[2020]
df_result[2050] = df_2050.reset_index()[2050]
df_result

,model,scenario,region,variable,unit,change,yearly change,2020,2050
0,SPM,Behaviour-focused,World,Decarbonization Rate (CO2),%,-114.868760,NaN,69.723183,-10.366972
1,SPM,Behaviour-focused,World,Decarbonization Rate (Kyoto Gases),%,-93.307517,8.619631,95.501730,6.391437
2,SPM,Behaviour-focused,World,Electrification Rate,%,130.533333,2.823202,0.190476,0.439111
3,SPM,Behaviour-focused,World,Energy Intensity,%,-75.855279,4.626569,0.005667,0.001368
4,SPM,Behaviour-focused,World,Food Demand|Crops (Share),%,9.756098,0.310783,82.000000,90.000000
5,SPM,Behaviour-focused,World,Land Cover|Forest and Other Natural Land,%,8.108108,0.260210,8140.000000,8800.000000
6,SPM,Behaviour-focused,World,Material Extraction,%,-17.035879,0.620606,62813.900000,52113.000000
7,SPM,Behaviour-focused,World,Primary Energy|Renewables (Share),%,543.521129,6.402564,7.612457,48.987768
8,SPM,Behaviour-focused,World,Yield|Cereal,%,23.268698,0.699758,3.610000,4.450000
9,SPM,Current Trends,World,Decarbonization Rate (CO2),%,-2.087050,0.070280,69.775475,68.319226
